<a href="https://colab.research.google.com/github/yeonmi02/mlproject/blob/main/%ED%94%84%EB%A1%AD%ED%96%87_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Prophet 모델 구조
데이터 분할:
시간 순서를 고려하여 데이터를 학습/검증용으로 나누었습니다. Shuffle 없이 시간 흐름을 유지하며 분할했습니다.

입력:
입력 변수는 Datetime(시간)과 예측 대상인 Total(전체 차량 수)로 구성되며, Prophet의 요구 형식에 맞춰 ds, y라는 컬럼명으로 변환했습니다.

모델 구성:
Prophet은 트렌드(추세), 계절성(일/주기), 휴일 효과를 내장 구성 요소로 자동 모델링합니다. 별도의 은닉층이나 뉴런은 존재하지 않습니다.

출력층:
출력은 예측하고자 하는 시계열 값(y, 예: Total 차량 수)으로, 지정한 미래 시점까지 연속적으로 예측됩니다.

추가 요소:
계절성(일간/주간) 자동 탐지, 이상치 대응 기능 포함. 필요시 외생 변수 추가 가능 (add_regressor()).

학습:
fit() 함수를 통해 전체 데이터를 기반으로 모델 학습을 진행했습니다.

In [ ]:
from prophet import Prophet
import matplotlib.pyplot as plt

# Prophet 입력 형식에 맞게 컬럼 이름 변경
df_prophet = traffic_2_30min[['Datetime', 'Total']].rename(columns={
    'Datetime': 'ds',
    'Total': 'y'
})

# 결측치 제거 (예측 실패 방지)
df_prophet.dropna(subset=['ds', 'y'], inplace=True)

# Prophet 모델 초기화 (일일 seasonality은 시간대별 패턴용, 주간도 포함)
model = Prophet(daily_seasonality=True, weekly_seasonality=True)

# 모델 학습
model.fit(df_prophet)

# 예측할 기간 생성 (예: 7일 = 30분 단위로 48*7 = 336 스텝)
future = model.make_future_dataframe(periods=336, freq='30min')

# 예측 수행
forecast = model.predict(future)

# 예측 결과 시각화
model.plot(forecast)
plt.title('교통 혼잡도(Total) 예측 - 30분 단위')
plt.show()

# 구성 요소별 시각화 (추세, 일간/주간 패턴 등)
model.plot_components(forecast)
plt.show()

NameError: name 'traffic_2_30min' is not defined

# Prophet 예측 결과 분석
###1.전체 예측 그래프 (맨 위)
검은 점: 실제 데이터 (y)

파란 선: Prophet의 예측값

하늘색 구간: 예측값의 신뢰 구간 (95%)

해석:

전체적으로 예측값이 실제값을 잘 따라가고 있음을 확인할 수 있습니다.

패턴이 주기적으로 반복되고 있어 Prophet이 시계열의 seasonality(계절성)을 잘 포착하고 있습니다.

오른쪽 부분은 미래 예측 구간으로, 신뢰 구간이 다소 넓어지는 것을 볼 수 있습니다. 이는 Prophet이 불확실성을 반영해준 것입니다.


-------------------------------------------------------------------------------

###2.추세(Trend) 그래프 (두 번째)
해석:

전체 데이터에서 전반적인 감소 추세가 나타나고 있습니다.

혼잡도 지수가 소폭 하락하고 있다는 의미이며, 이 데이터 구간에서는 시간이 지남에 따라 혼잡도가 완만하게 줄어드는 경향이 있습니다

-------------------------------------------------------------------------------
###3.주간 seasonality 그래프 (세 번째)
X축: 요일 (Sunday ~ Saturday)

Y축: 혼잡도의 상대적인 증감량

해석:

금요일과 토요일에 혼잡도가 크게 증가, 월~수는 상대적으로 낮음.

이는 주말 및 금요일 야간 교통량이 많은 지역 특성을 반영한 것으로 보입니다.

Prophet이 요일별 트렌드를 잘 파악하고 있다는 의미입니다.

-------------------------------------------------------------------------------
###4.일간 seasonality 그래프 (네 번째)
X축: 하루 24시간 시간대

Y축: 혼잡도의 상대적인 증감량

해석:

**출근 시간**과 **퇴근 시간**에 혼잡도가 급격히 증가합니다.

**심야 시간대(자정 ~ 새벽 5시)**에는 혼잡도가 낮습니다.

이는 일반적인 교통 패턴과 일치하며, Prophet이 일일 주기를 잘 학습한 결과입니다.

-------------------------------------------------------------------------------
전반적 추세	혼잡도는 전체적으로 완만하게 감소


주간 패턴	금·토요일에 교통량 급증, 평일은 상대적으로 낮음


일간 패턴	출퇴근 시간에 혼잡도 급증, 새벽 시간대는 낮음


모델 신뢰성	실제 데이터와 예측값의 일치도가 높고, 주기성 패턴을 정확하게 반영함

-------------------------------------------------------------------------------

Prophet 모델을 통해 예측한 결과, 데이터는 주간 및 일간 주기성을 강하게 보이며, 특히 금요일과 토요일에 교통량이 급증하고 평일 낮 시간에는 감소하는 경향이 관측되었습니다. 또한 시간대별로는 출퇴근 시간 혼잡도가 높게 나타났으며, 이는 실제 교통 흐름과 잘 일치함을 보여줍니다.

# 2. Random Forest 모델 구조 (회귀/분류)
데이터 분할:
시간 순서를 유지하며 학습용과 테스트용으로 분리했습니다 (예: 8:2 비율). 회귀와 분류 모두 동일한 방식 사용.


입력:
수치형 및 원-핫 인코딩된 범주형 변수들을 모두 입력 특성으로 사용했습니다. 예: 요일, 차량 종류, 교통 상황 등.


모델 구성:
여러 개의 결정 트리를 앙상블하여 결과를 도출합니다.

회귀: 각 트리의 예측 평균을 출력

분류: 다수결로 클래스 결정

출력층
ㄴ회귀: 연속형 값 예측 (예: Total 차량 수)

ㄴ분류: 클래스 예측 (예: Traffic Situation 상태 분류)

활성화 함수:
결정 트리 기반 모델이라 별도의 활성화 함수는 사용되지 않습니다.

옵티마이저:
트리 기반 모델은 옵티마이저 개념이 없으며, 내부적으로는 분할 기준(gini, entropy, mse 등)을 사용합니다.

손실 함수:

회귀: 평균 제곱 오차(MSE)

분류: 지니 불순도 또는 엔트로피

하이퍼파라미터:
트리 개수(n_estimators=100), 최대 깊이, 최소 샘플 수 등 조정 가능. 이번에는 기본값 또는 적절한 설정을 사용했습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 예측 대상 변수 (y)와 특성 (X) 지정
target_col = 'Total'
feature_cols = [col for col in traffic_2_30min.columns if col not in ['Datetime', target_col]]

X = traffic_2_30min[feature_cols]
y = traffic_2_30min[target_col]

# 훈련/테스트 데이터 나누기 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 랜덤 포레스트 회귀 모델 생성 및 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R2 Score: {r2:.4f}')

# 실제 vs 예측 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
plt.plot(y_test.values[:100], label='Actual', marker='o')
plt.plot(y_pred[:100], label='Predicted', marker='x')
plt.title('Random Forest Prediction (예측 vs 실제)')
plt.xlabel('Time Index')
plt.ylabel('Total Vehicle Count')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'traffic_2_30min' is not defined

In [ ]:
import pandas as pd

# Assuming the data is in a CSV file named 'traffic_data.csv'.
# Please replace 'traffic_data.csv' with the actual file name if it's different.
try:
  traffic_2_30min = pd.read_csv('traffic_data.csv')
except FileNotFoundError:
  print("Error: 'traffic_data.csv' not found. Please replace with the correct file name.")

Error: 'traffic_data.csv' not found. Please replace with the correct file name.


#Random Forest 예측 결과 분석
###1.그래프
파란 점 (Actual): 실제 관측된 차량 수 (y_test)

주황색 선 (Predicted): Random Forest가 예측한 차량 수 (y_pred)

X축: 시간 인덱스 (순서대로 정렬된 시간)

Y축: 총 차량 수 (Total Vehicle Count)

--------------------------------------------------------------------------------

##분석 요약

**예측 정확도**: 실제값과 예측값이 거의 일치함. 특히 차량 수의 급격한 변화 구간에서도 잘 따라감.

**변동성 대응**:	차량 수가 급격히 오르내리는 지점에서도 예측이 빠르게 반응함 → Random Forest의 강점


**과적합 여부**:	데이터셋에 따라 과적합 가능성도 있지만, 이 그래프에서는 실제값과 예측값이 안정적으로 일치함


**오차 구간**:	일부 데이터에서 미세한 오차는 존재하지만 전체적인 예측 경향은 매우 우수함

--------------------------------------------------------------------------------
##종합 분석
Random Forest 회귀 모델은 실제 차량 수 변화와 높은 일치도를 보이며, 급격한 차량 수 변동 구간에서도 안정적인 예측 성능을 보여주었다. 예측값은 실제값을 거의 정확히 따라가며, 시계열 패턴을 잘 포착하였다. 따라서 해당 모델은 교통량과 같은 복잡한 비선형 데이터를 예측하는 데 적합한 성능을 보인다.